# Cartographie des Musées de France : Une Exploration Géographique et Culturelle

La présente analyse se concentre sur la cartographie des musées de France en utilisant un jeu de données provenant du site data.culture.gouv.fr. Le jeu de données utilisé est disponible à l'adresse https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin et fournit des informations détaillées sur les musées de France, y compris leur localisation, coordonnées géographiques et autres détails administratifs.


L'objectif de cette analyse est de préparer les données pour une visualisation sous Folium, permettant de créer une carte interactive des musées de France avec des infobulles contenant des informations pertinentes, notamment un lien vers Street View. Cela facilitera l'exploration géographique et culturelle des musées, offrant une perspective unique sur la répartition et les caractéristiques de ces institutions à travers le pays.


## Méthodologie

La méthodologie employée dans cette analyse implique plusieurs étapes clés. 
Tout d'abord, les données sont extraites du jeu de données source, puis nettoyées et transformées à l'aide de requêtes SQL via DuckDB pour ne retenir que les informations nécessaires et pertinentes pour la visualisation.
Ensuite, les données nettoyées sont utilisées pour créer une carte interactive sous Folium, avec des marqueurs représentant les musées, accompagnés d'infobulles contenant le nom du musée, son adresse et un lien vers Street View.
Cette approche permet une visualisation efficace et une exploration interactive des données, facilitant l'identification de modèles et de tendances dans la répartition géographique des musées de France.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "identifiant_museofile" AS id_musee,
  "nom_officiel_du_musee" AS nom_musee,
  "adresse" AS adresse_musee,
  "code_postal" AS code_postal,
  "latitude" AS lat,
  "longitude" AS lon,
  ST_AsText("geolocalisation") AS geom_wkt,
  CONCAT('https://www.google.com/maps/streetview?location=', "latitude", ',', "longitude") AS street_view_url,
  CONCAT('<a href="', "url", '">Site web du musée</a>') AS url_musee_html
FROM 
  loaded_dataset
WHERE 
  "latitude" IS NOT NULL AND "longitude" IS NOT NULL
ORDER BY 
  id_musee """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 1217 lignes


,id_musee,nom_musee,adresse_musee,code_postal,lat,lon,geom_wkt,street_view_url,url_musee_html
0,M0001,musée de la Folie Marco,"30, rue du docteur Sultzer",67140,48.410166,7.451102,POINT (7.451102 48.410166),https://www.google.com/maps/streetview?locatio...,"<a href=""www.musee-foliemarco.com"">Site web du..."
1,M0002,musée de la poterie,2 rue de Kuhlendorf,67660,48.900348,7.914409,POINT (7.914409 48.900348),https://www.google.com/maps/streetview?locatio...,"<a href=""www.betschdorf.com/vie-culturelle-et-..."
2,M0003,musée du pays de Hanau - histoire et vies d'un...,3 place du Château,67330,48.824977,7.482915,POINT (7.482915 48.824977),https://www.google.com/maps/streetview?locatio...,"<a href=""www.museedupaysdehanau.eu"">Site web d..."
3,M0004,musée alsacien,"1, place Joseph Thierry",67500,48.814611,7.789527,POINT (7.789527 48.814611),https://www.google.com/maps/streetview?locatio...,"<a href=""www.ville-haguenau.fr/musee-alsacien""..."
4,M0005,musée historique,"9, rue du Maréchal Foch",67500,48.812831,7.791481,POINT (7.791481 48.812831),https://www.google.com/maps/streetview?locatio...,"<a href=""www.ville-haguenau.fr/musee-historiqu..."


## 📈 Visualisation

La bibliothèque principale utilisée est Folium, qui permet de créer des cartes interactives en combinant les capacités de Leaflet.js avec la simplicité de Python. Folium est adaptée pour représenter des données géolocalisées, comme ici des musées avec leurs adresses et coordonnées, en permettant une visualisation intuitive et interactive. Cela permet une exploration facile des données sur une carte.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import re

dataviz = folium.Map(location=[46.227638, 2.213749], zoom_start=6, tiles='CartoDB positron')

cluster = MarkerCluster(name="musées", overlay=True, control=True).add_to(dataviz)

df_clean = df.dropna(subset=['lat', 'lon', 'nom_musee', 'adresse_musee', 'street_view_url'])
df_clean = df_clean.sort_values('nom_musee')

for _, row in df_clean.iterrows():
    html = f"""
    <b style="font-family:Arial;font-size:14px">{row['nom_musee']}</b><br>
    <span style="font-family:Arial;font-size:12px">{row['adresse_musee']}</span><br>
    <a href="{row['street_view_url']}" target="_blank" style="font-family:Arial;font-size:12px">Street View</a>
    """
    popup = folium.Popup(folium.IFrame(html, width=250, height=100), max_width=250)
    folium.Marker([row['lat'], row['lon']], popup=popup, tooltip=row['nom_musee'],
                  icon=folium.Icon(color='blue', icon='fa-museum', prefix='fa')).add_to(cluster)

folium.LayerControl(collapsed=False).add_to(dataviz)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250817_214742.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250817_214742.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Folium. Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250817_214742.html


--> Création du marqueur de capture d'écran.
